In [1]:
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering,KMeans
from scipy.cluster.hierarchy import dendrogram, linkage

In [5]:
import psycopg2
from sqlalchemy import create_engine,MetaData,Table
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import exp,log,sqrt
from scipy.stats import norm
from sklearn.preprocessing import scale,StandardScaler
from datetime import timedelta

In [32]:
class MCA():
    
    def __init__(self, n_components):
        self.n_components = n_components
        
    def generer_matrice(self,donnee_new):
        length=[]
        for i in range(len(donnee_new[0])):
            length.append(len(set(donnee_new[:,i])))
        matrice=[]
        
        for i in range(len(donnee_new)):
            y=[]
        # range(1,len(donnee_new[0])) starts from 1 becaouse the colonne is station_id, not useful for me
            for j in range(0,len(donnee_new[0])):
                x=length[j]*[0]
                exist=sorted(list(set(donnee_new[:,j])))
                ind= exist.index(int(donnee_new[i,j]))
                x[ind]=1
                y=np.append(y,x)
            matrice.append(y)
        return matrice
    
    def fit(self,donnee_new):
        length=[]
        for i in range(len(donnee_new[0])):
            length.append(len(set(donnee_new[:,i])))
        matrice=[]
        for i in range(len(donnee_new)):
            y=[]
            for j in range(0,len(donnee_new[0])):
                x=length[j]*[0]
                exist=sorted(list(set(donnee_new[:,j])))
                ind= exist.index(int(donnee_new[i,j]))
                x[ind]=1
                y=np.append(y,x)
            matrice.append(y)
        pk=np.sum(matrice,axis=0)/len(matrice)
        I=len(matrice)
        J=len(donnee_new[0])
        pk_inverse=[1/pk[i] for i in range(len(pk))]
        pk_racine=[sqrt(pk[i]) for i in range(len(pk))]
        pk_matrice_inverse=np.diag(pk_inverse)
        pk_matrice_racine=np.diag(pk_racine)
        matrice_tr=np.dot(matrice,pk_matrice_inverse)
        for i in range(len(matrice_tr)):
            for j in range(len(matrice_tr[0])):
                matrice_tr[i,j]=matrice_tr[i,j]-1
        matrice_tr2=np.dot(matrice_tr,pk_matrice_racine)
        matrice_tr3=matrice_tr2/sqrt(I*J)
        pca = PCA(n_components=self.n_components)
        pca.fit(matrice_tr3)
        self.explained_variance_ = pca.explained_variance_
        self.components_ = pca.components_
        self.explained_variance_ratio_=pca.explained_variance_ratio_
        
    def explained_variance_(self):
        return self.explained_variance_
    
    def components_(self):
        return self.components
    def explained_variance_ratio_(self):
        return self.explained_variance_ratio_
 


In [33]:
mca=MCA(n_components=2)

In [34]:
x=np.array([[0,1,2],
            [1,0,1],
            [0,1,0],
            [0,0,1],
            [1,1,2]])

In [35]:
matrice=mca.generer_matrice(x)

In [36]:
print (matrice)

[array([1., 0., 0., 1., 0., 0., 1.]), array([0., 1., 1., 0., 0., 1., 0.]), array([1., 0., 0., 1., 1., 0., 0.]), array([1., 0., 1., 0., 0., 1., 0.]), array([0., 1., 0., 1., 0., 0., 1.])]


In [37]:
mca.fit(x)

In [38]:
print (mca.explained_variance_)

[0.17165137 0.11111111]


In [39]:
print (mca.components_)

[[-0.15499413  0.18982827  0.52891843 -0.43186009 -0.31969727  0.52891843
  -0.30285832]
 [-0.41039134  0.50262469 -0.12565617  0.10259784 -0.53311399 -0.12565617
   0.50262469]]


In [40]:
print (mca.explained_variance_ratio_)

[0.51495412 0.33333333]
